In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [98]:
# get : 서버에서 데이터 받아오기

In [4]:
# requests.get            # requests.get # 받아온 데이터 확인 세트임
response = requests.get("https://finance.naver.com/")  
response

<Response [200]>

In [5]:
# 받아온 데이터 확인 
html = response.text 

# 뷰티풀숲으로 데이터 파싱(정제, 추출) 
soup = bs(html, 'html.parser') 
soup   # html 이 깔끔하게 정리되서 출력됨. 


<html lang="ko">
<head>
<title>Npay 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="Npay 증권" name="apple-mobile-web-app-title">
<meta content="Npay 증권" property="og:title">
<meta content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png" property="og:image"/>
<meta content="https://finance.naver.com" property="og:url"/>
<meta content="국내 해외 증시 지수, 시장지표, 뉴스, 증권사 리서치 등 제공" property="og:description"/>
<meta content="article" property="og:type"/>
<meta content="" property="og:article:thumbnailUrl"/>
<meta content="Npay 증권" property="og:article:author"/>
<meta content="http://FINANCE.NAVER.COM" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20260121155431/css/finance_header.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/st

# 뉴스 제목 크롤링

In [ ]:
# 증권 - f12 -제목 뉴스 클릭-코드클릭위 우클릭 카피 -카피셀렉트(복사)

In [6]:
# 첫번째 뉴스 기사 제목    
selector = "#content > div.article > div.section > div.news_area._replaceNewsLink > \
div > ul > li:nth-child(1) > span > a"
tag = soup.select_one(selector)  
tag.text

'트럼프 관세 충격, 버티는 코스피…"변동성 커질수 있어 유의해야"'

In [7]:
# 두번째 뉴스 기사 제목    
selector = "#content > div.article > div.section > div.news_area._replaceNewsLink > \
div > ul > li:nth-child(2) > span > a"
tag = soup.select_one(selector)
tag.text

"'1000스닥 못 참지' 6억원 역대급 자금 유입…'이것' 개미들 쏠렸다"

# Top 종목

In [8]:
# top 첫번째 종목 이름
selector = "#nxt_topItems1 > tr:nth-child(1) > th > a"
top1 = soup.select_one(selector)
top1.text

'한전산업'

In [9]:
# 가격
selector = "#nxt_topItems1 > tr:nth-child(1) > td:nth-child(2)"
top1_price = soup.select_one(selector)
top1_price = top1_price.text
top1_price

'15,280'

In [10]:
# 상승/하락
selector = "#nxt_topItems1 > tr:nth-child(1) > td:nth-child(3)"
top1_updown = soup.select_one(selector)
top1_updown = top1_updown.text
top1_updown

'상승 3,010'

In [11]:
# 등락률(%)
selector = "#nxt_topItems1 > tr:nth-child(1) > td:nth-child(4)"
top1_ratio = soup.select_one(selector)
top1_ratio = top1_ratio.text.strip()   
# strip() : 문자열 공백 제거 
top1_ratio

'+24.53%'

In [107]:
# 공백들
# \n : 줄바꿈 
# \t : tab 

# 반복문으로 일괄 데이터 수집

In [108]:
# 각 종목명, 가격, 상승하락, 등락률을 각각 리스트로 저장하기
# 리스트들을 데이터프레임으로 변환하기

In [3]:
# 총 15개 종목들을 for문 돌면서 각 데이터 수집하기
#f" " 는 f-string (포맷 문자열) 이라는 뜻이야.👉 문자열 안에 변수나 계산식을 바로 넣을 수 있게 해주는 문법이야. {}
stock_list = []
price_list = []
updown_list = []
ratio_list = []

for i in range(1, 16):
    # 종목명 리스트로 저장
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > th > a"
    top_stock = soup.select_one(selector)
    top_stock = top_stock.text
    stock_list.append(top_stock)
    # print(top_stock)

    # 가격 리스트로 저장
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > td:nth-child(2)"
    top_price = soup.select_one(selector)
    top_price = top_price.text
    if ',' in top_price:
        top_price = int(top_price.replace(",", ""))
    price_list.append(top_price)
    #print(top_price)

    # 상승/하락 리스트로 저장
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > td:nth-child(3)"
    top_updown = soup.select_one(selector)
    top_updown = top_updown.text
    updown_list.append(top_updown)
    # print(top_updown)

    # 등락률 리스트로 저장
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > td:nth-child(4)"
    top_ratio = soup.select_one(selector)
    top_ratio = top_ratio.text.strip()   
    # strip() : 문자열 공백 제거 
    ratio_list.append(top_ratio)
    # print(top_ratio)

NameError: name 'soup' is not defined

In [120]:
# 가격 평균
import numpy as np
np.mean(price_list)

42553.0

In [110]:
import pandas as pd
# 수집된 리스트들을 데이터프레임으로 바꾸기
# pd.DataFrame({'컬럼명1':리스트1, '컬럼명2':리스트2})

df_stock = pd.DataFrame({'종목명':stock_list, '가격':price_list, '상승하락':updown_list, '등락률':ratio_list})
df_stock

,종목명,가격,상승하락,등락률
0,씨아이에스,"10,630","상승 1,510",+16.56%
1,NHN KCP,"20,150","상승 1,660",+8.98%
2,한화생명,"3,555",상승 115,+3.34%
3,카카오페이,"69,900","상승 2,100",+3.10%
4,에코프로,"130,600","상승 24,700",+23.32%
5,삼성전자,"152,400",상승 300,+0.20%
6,레이크머티리얼즈,"24,750","상승 4,450",+21.92%
7,미래에셋벤처투자,"15,650","상승 2,420",+18.29%
8,고영,"31,900","상승 3,000",+10.38%
9,한농화성,"24,900","상승 1,500",+6.41%


In [111]:
# 데이터프레임 -> csv 파일로 저장
df_stock.to_csv("stock_top.csv", encoding='utf-8-sig')

In [113]:
# 데이터프레임 -> 엑셀 파일로 저장
df_stock.to_excel("stock_top.xlsx")

In [1]:
stock_list = []
price_list = []
updown_list = []
ratio_list = []

for i in range(1, 16):
    # 종목명
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > th > a"
    top_stock = soup.select_one(selector)
    top_stock = top_stock.text
    stock_list.append(top_stock)
    # print(top_stock)

    # 가격 
    selector = f"#nxt_topItems1 > tr:nth-child({i}) > td:nth-child(2)"
    top_price = soup.select_one(selector)
    top_price = top_price.text
    price_list.append(top_price)
    # print(top_price)

    

NameError: name 'soup' is not defined